In [1]:
import tweepy
import pandas as pd
import os
import unidecode
#import datetime
from datetime import timedelta, datetime 
from dateutil.relativedelta import relativedelta
import time
import datetime as dt 
import pymongo
from pymongo import MongoClient
import numpy as np
pd.options.mode.chained_assignment = None 

In [28]:
trendings_keywords = pd.read_csv('../Data/trendings.csv')
trendings_keywords['Sector'].value_counts()

Neutro       91
Derecha      42
Izquierda    40
Name: Sector, dtype: int64

In [10]:
trendings_keywords[['trending','Sector']]

,trending,Sector
0,#NosUnimosONosJodemos,Derecha
1,#FedericoEsColombia,Derecha
2,#LoPeorDeEstasElecciones,Neutro
3,#MiVotoEsPetroYFrancia,Izquierda
4,#MiVotoEsPetroYFrancia,Izquierda
...,...,...
168,#DePetroEspero,Neutro
169,#PetroNecesita,Neutro
170,#EsMuyImportante,Neutro
171,#PetroBastaYa,Neutro


In [57]:
df_list = []
for i in range(len(trendings_keywords)):
    start_date = trendings_keywords['date'][i]
    newpath = '../Data/Trendings/'+ start_date + '/'
    trending = trendings_keywords['trending'][i]
    file_name = start_date + '__' +  trending + '.gzip'
    trendings_df = pd.read_parquet(newpath + file_name)
    trendings_df['Trending'] =  trending 
    df_list.append(trendings_df)
full_df = pd.concat(df_list)
full_df = full_df.reset_index(drop=True)
full_df.to_parquet('../Data/Full_dataset/full_data.gzip',
          compression='gzip')

In [2]:
full_df = pd.read_parquet('../Data/Full_dataset/full_data.gzip')

In [92]:
def cleaning(df, prop, words):
    df['Number_words'] = df['text'].str.count(' ') + 1
    df['Number_non_text'] = df['text'].str.count('#|@|http')
    df['non_text_proportion'] = df['Number_non_text']/df['Number_words']
    df = df[(df['non_text_proportion'] <= prop) & (df['Number_words'] > words)]
    df = df.reset_index(drop=True)
    return df

In [95]:
cleaned = cleaning(full_df, 0.2, 5)
cleaned.to_parquet('../Data/Full_dataset/data_filtered.gzip',
          compression='gzip')

In [6]:
cleaned = pd.read_parquet('../Data/Full_dataset/data_filtered.gzip')

In [30]:
cleaned_political = cleaned.merge(trendings_keywords[['trending','Sector']].drop_duplicates(), how='left', left_on='Trending', right_on='trending')

In [31]:
cleaned_political['Sector'].value_counts()

Neutro       152594
Izquierda     86006
Derecha       52235
Name: Sector, dtype: int64